### DESCRIPTION:
    Create a prompt to extract entities by turning a user’s question into a JSON object that can be used to search in a Database

### REQUIREMENTS:
    Create an .env file with your OpenAI API key and save it in the root directory of this project with the following
    OPENAI_DEPLOYMENT_ENDPOINT ="<your openai endpoint>" 
    OPENAI_API_KEY = "<your openai api key>"
    OPENAI_DEPLOYMENT_NAME = "<your gpt35 deployment name>"
    OPENAI_DEPLOYMENT_VERSION = "<gpt35 api version>"
    OPENAI_MODEL_NAME="<gpt35 model name>"


#### Call OpenAI to generate JSON from user's input ####

In [4]:
from dotenv import load_dotenv
import pandas as pd
import os
import openai
import utils
import json

In [20]:
json_template_prefix = """
<|im_start|>system
You are an assistant designed to extract entities from a user question. Users will paste in a string of text and you will respond with entities you have extracted from the text as a JSON object. Here is an example of your output format:
{moods:[],instruments:[],genres:[], suggestions:mood:[],instrument:[],genre:[]}  
"""
template_sufix = "<|im_end|>\n<|im_start|>assistant"

In [73]:
def call_openai(template_prefix, text):
    prompt = template_prefix + text + template_sufix
    response = openai.Completion.create(
        engine=utils.OPENAI_DEPLOYMENT_NAME,
        prompt=prompt,
        temperature=0,
        max_tokens=4096,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["<|im_end|>"])
    response = response['choices'][0]['text']
    response = utils.remove_chars("\n", response)
    response=utils.start_after_string("Answer:", response)
    response=utils.remove_tail_tags("<|im_end|>", response)
    return response

def call_openai_json_response(template_prefix, text):
    response = call_openai(template_prefix, text)
    utils.pretty_print_json_string(response)

In [48]:
call_openai_json_response(json_template_prefix, "I would like to find music for guitar or piano, for a happy mood")

{
  "moods": [
    "happy"
  ],
  "instruments": [
    "guitar",
    "piano"
  ],
  "genres": [],
  "suggestions": {
    "mood": [
      "upbeat",
      "cheerful"
    ],
    "instrument": [],
    "genre": []
  }
}
